# Testing fs8

In [19]:
Threads.nthreads()

1

In [20]:
using LinearAlgebra
using Turing
using LimberJack
using GaussianProcess
using PythonCall
using NPZ
using BenchmarkTools;

In [21]:
np = pyimport("numpy");

In [22]:
meta = np.load("../data/DESY1/gcgc_gcwl_wlwl_meta.npz")
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
cls_data = pyconvert(Vector{Float64}, meta["cls"])
cls_cov = pyconvert(Matrix{Float64}, meta["cov"])
cls_inv_cov = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [23]:
files = npzread("../data/DESY1/gcgc_gcwl_wlwl_files.npz");

In [24]:
fs8_meta = npzread("../data/fs8s/fs8s.npz")
fs8_zs = fs8_meta["z"]
fs8_data = fs8_meta["data"]
fs8_cov = fs8_meta["cov"]

11×11 Matrix{Float64}:
 0.00203355   0.000811829  0.000264615  …  0.0     0.0     0.0       0.0
 0.000811829  0.00142289   0.000662824     0.0     0.0     0.0       0.0
 0.000264615  0.000662824  0.00118576      0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0          …  0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.0       0.0
 0.0          0.0          0.0             0.0144  0.0     0.0       0.0
 0.0          0.0          0.0             0.0     0.0121  0.0       0.0
 0.0          0.0          0.0             0.0     0.0     0.003025  0.0
 0.0          0.0          0.0          …  0.0     0.0     0.0       0.013456

In [25]:
cov_tot = zeros(Float64, length(fs8_data)+length(cls_data), length(fs8_data)+length(cls_data))
cov_tot[1:length(fs8_data), 1:length(fs8_data)] = fs8_cov
cov_tot[length(fs8_data)+1:(length(fs8_data)+length(cls_data)),
        length(fs8_data)+1:(length(fs8_data)+length(cls_data))] = cls_cov
data_vector = [fs8_data ; cls_data];

In [26]:
fid_cosmo = Cosmology()
N = 100
latent_x = Vector(0:0.3:3)
x = Vector(range(0., stop=3., length=N));

In [27]:
pars = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
         1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
        -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
         2.389208e-01,   -6.435288e-01, 
         1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
        -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
         8.536883e-01,    2.535825e-01];

In [28]:
nuisances = Dict("DESgc__0_0_b" => pars[5],
                 "DESgc__1_0_b" => pars[6],
                 "DESgc__2_0_b" => pars[7],
                 "DESgc__3_0_b" => pars[8],
                 "DESgc__4_0_b" => pars[9],
                 "DESgc__0_0_dz" => pars[10],
                 "DESgc__1_0_dz" => pars[11],
                 "DESgc__2_0_dz" => pars[12],
                 "DESgc__3_0_dz" => pars[13],
                 "DESgc__4_0_dz" => pars[14],

                 "A_IA" => pars[15],
                 "alpha_IA" => pars[16],

                 "DESwl__0_e_dz" => pars[21],
                 "DESwl__1_e_dz" => pars[22],
                 "DESwl__2_e_dz" => pars[23],
                 "DESwl__3_e_dz" => pars[24],
                 "DESwl__0_e_m" => pars[17],
                 "DESwl__1_e_m" => pars[18],
                 "DESwl__2_e_m" => pars[19],
                 "DESwl__3_e_m" => pars[20])

Dict{String, Float64} with 20 entries:
  "DESwl__2_e_dz" => -0.00109794
  "DESwl__3_e_m"  => -0.00111773
  "A_IA"          => 0.238921
  "DESwl__0_e_m"  => 0.00180272
  "DESwl__3_e_dz" => -0.00491232
  "DESgc__3_0_dz" => 0.00119705
  "DESgc__1_0_b"  => 1.63905
  "DESgc__2_0_b"  => 1.59717
  "DESwl__1_e_dz" => 0.00677778
  "DESwl__1_e_m"  => -0.00550899
  "DESgc__4_0_dz" => -0.0051998
  "DESgc__3_0_b"  => 1.94458
  "DESwl__2_e_m"  => 0.0195251
  "DESgc__1_0_dz" => -0.00284
  "DESgc__0_0_dz" => -0.00467938
  "DESgc__4_0_b"  => 2.00725
  "alpha_IA"      => -0.643529
  "DESgc__0_0_b"  => 1.34389
  "DESwl__0_e_dz" => -0.0174408
  "DESgc__2_0_dz" => 0.00177157

## Sample Prior

In [33]:
function make_theory(Ωm, v;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files,
                      fid_cosmo=fid_cosmo,
                      latent_x=latent_x,
                      x=x)
    

    Ωm = pars[26]
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    s8 = 0.811

    eta = 0.02
    l = 0.3

    latent_N = length(latent_x)
    
    mu = fid_cosmo.Dz(vec(latent_x))
    K = sqexp_cov_fn(latent_x; eta=eta, l=l)
    latent_gp = latent_GP(mu, v, K)
    gp = conditional(latent_x, x, latent_gp, sqexp_cov_fn;
                      eta=eta, l=l)

    dmu =  growth_rate(fid_cosmo, vec(latent_x)) ./ (1 .+ latent_x)
    dK = sqexp_cov_grad(latent_x; eta=eta, l=l)
    latent_dgp = latent_GP(dmu, v, dK)
    dgp = conditional(latent_x, x, latent_dgp, sqexp_cov_grad;
                      eta=eta, l=l)
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit";
                                     custom_Dz=[x, gp, dgp])
    
    cls = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    fs8s = fs8(cosmology, x)
    
    return latent_gp, fs8s, cls
end;

In [34]:
N_samples = 100
cls_preds = zeros(N_samples, length(cls_data))
gps_preds = zeros(N_samples, length(latent_x))
fs8s_preds = zeros(N_samples, length(x))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    v = rand(MvNormal(zeros(length(latent_x)), ones(length(latent_x))))
    gps_preds[i,:], fs8s_preds[i,:], cls_preds[i,:] = make_theory(Ωm, v)
end

In [35]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);
fs8s_preds_m, fs8s_preds_s = mean(fs8s_preds, dims=1), std(fs8s_preds, dims=1);
gps_preds_m, gps_preds_s = mean(gps_preds, dims=1), std(gps_preds, dims=1);

In [36]:
npzwrite("fs8_ploting.npz", Dict("cls_m" => cls_preds_m,
                                 "cls_s" => cls_preds_s, 
                                 "fs8s_m" => fs8s_preds_m,
                                 "fs8s_s" => fs8s_preds_s,
                                 "gps_m" => gps_preds_m,
                                 "gps_s" => gps_preds_s)) 

## Sample Posterior

In [41]:
@model function model(data_vector;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files,
                      fid_cosmo=fid_cosmo,
                      latent_x=latent_x,
                      x=x)

    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb = 0.05 # ~ Uniform(0.028, 0.065)
    h = 0.67 #~ Uniform(0.64, 0.82)
    ns = 0.96 #~ Uniform(0.84, 1.1)
    s8 = 0.811
    
    eta = 0.05
    l = 1
    latent_N = length(latent_x)
    v ~ filldist(truncated(Normal(0, 1), -3, 3), latent_N)
    
    mu = fid_cosmo.Dz(vec(latent_x))
    K = sqexp_cov_fn(latent_x; eta=eta, l=l)
    latent_gp = latent_GP(mu, v, K)
    gp = conditional(latent_x, x, latent_gp, sqexp_cov_fn;
                      eta=eta, l=l)

    dmu =  growth_rate(fid_cosmo, vec(latent_x)) ./ (1 .+ latent_x)
    dK = sqexp_cov_grad(latent_x; eta=eta, l=l)
    latent_dgp = latent_GP(dmu, v, dK)
    dgp = conditional(latent_x, x, latent_dgp, sqexp_cov_grad;
                      eta=eta, l=l)
    
    cosmology = Cosmology(Ωm, Ωb, h, ns, s8,
                          tk_mode="EisHu",
                          Pk_mode="Halofit", 
                          custom_Dz=[x, gp, dgp])
    
    cls = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    fs8s = fs8(z_fs8)
    theory = [cls; fs8s]
    
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [42]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

LoadError: UndefVarError: z_fs8 not defined

In [40]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: UndefVarError: nadapts not defined